# [Subsets Sum](https://cses.fi/problemset/task/1628/)
- Given
    + `N`: Number of items
    + `X`: Target sum
    + `T[i]`: item value
- Output
    + The number of ways (combinations) of `T[i]` can sum to `X`

```
N X
T[0] T[1] T[2] ... T[N-1]
```

#### Constraints
- $1 \leq N \leq 40$
- $1 \leq X \leq 10^9$
- $1 \leq T[i] \leq 10^9$

#### Example

```
# Input
4 5
1 2 3 2

# Output
3

# Explanation
[0]1 [1]2
[0]1 [3]2
[2]3
```

#### Solution $O(2^{N/2}N)$
- Bruteforce: $O(2^N)$
    + If N max = 40 -> too large
    + If N max = 20 -> feasible
- Use **Meet in the middle** to reduce the bruteforce complexity from $O(2^N)$ to $O(2^{N/2})$
- Other optimization
    + Processing time O(N) can be reduced by sorting T decreasing -> large number considered 1st -> Prune and early stop the search
    
#### Code

```C++
int N, X;
vector<int> T;


unordered_map<int, int> cnt;
int ans = 0;
vector<int> subset;
void iterate_subsets(int idx, const vector<int> &A, int task) {
    if(idx == A.size()) {
        // do task left half
        if(task == 0) {
            int sum_left = 0;
            for(int &num: subset) {
                sum_left += num;

                // Early stoping
                if(sum_left > X) return;
            }
            if(sum_left <= X) cnt[sum_left] += 1;
        }

        // do task right half
        else {
            int sum_right = 0;
            for(int &num: subset) {
                sum_right += num;

                // Early stoping
                if(sum_right > X) return;
            }
            if(sum_right <= X) ans += cnt[X - sum_right];
        }
        return;
    }

    // Add T[i]
    subset.push_back(A[idx]);
    iterate_subsets(idx+1, A, task);
    subset.pop_back();

    // Not add T[i]
    iterate_subsets(idx+1, A, task);
}



int solve() {
    ans = 0;

    // Sort T decreasing
    sort(T.begin(), T.end(), greater<int>());

    // Iterate left half subsets
    vector<int> left_half = vector<int>(T.begin(), T.begin() + N/2);
    iterate_subsets(0, left_half, 0);

    // Iterate right half subsets
    vector<int> right_half = vector<int>(T.begin() + N/2, T.end());
    iterate_subsets(0, right_half, 1);

    return ans;
}
```

# [4-Sum](https://cses.fi/problemset/task/1642/)

- Given
    + `N`: Number of items
    + `K`: Target sum
    + `A[i]`
- Output
    + Print a solution (indices) of 4 numbers that sum to K. **Note**: All indices must be distinct
    + If no sol: print `IMPOSSIBLE`

```
N K
A[0] A[1] A[2] ... A[N-1]
```

#### Constraints
+ $1 \leq N \leq 1000$


#### Example

```
# Input
8 15
3 2 5 8 1 3 2 3

# Output
2 4 6 7

# Explanation
[2]2 + [4]8 + [6]3 + [7]2 = 15
```

#### Solution $O(N^2)$
- Use meet-in-the-middle reduce 4-SUM to 2-SUM


```C++
struct Object {
    int idx;
    int value;
};

int N, K;
vector<Object> A;


void solve() {
    // Sort increasing for pruning
    sort(A.begin(), A.end(),
    [](const Object &a, const Object &b)
        {return a.value < b.value;});

    // Pruning 1: Max val < K/4
    if(A[N-1].value < K / 4) {
        cout << "IMPOSSIBLE" << endl;
        return;
    }

    unordered_map<int, pair<int,int>> first_half_2_sum;
    for(int x=0; x<N; ++x) {
        // 2-Sum for left half
        for(int i=0; i<x; ++i) {
            int left_half_2_sum = A[x].value + A[i].value;

            // Pruning 2
            if(left_half_2_sum > K) break;

            first_half_2_sum.insert( {left_half_2_sum, {A[i].idx, A[x].idx}} );
        }

        // 2-Sum for right half
        for(int j=x+1; j<N; ++j) {
            int right_half_2_sum = A[x].value + A[j].value;
            int missing = K - right_half_2_sum;

            // Pruning 3
            if(right_half_2_sum > K) break;

            if(first_half_2_sum.count(missing) == true && \
                first_half_2_sum[missing].first != A[x].idx && \
                first_half_2_sum[missing].second != A[x].idx) {

                // Result
                cout << first_half_2_sum[missing].first << ' ' \
                    << first_half_2_sum[missing].second << ' ' \
                    << A[x].idx << ' ' \
                    << A[j].idx << endl;
                return;
            }
        }
    }

    // No solutions
    cout << "IMPOSSIBLE" << endl;
    return;
}
```